# 1. 讀取檔案

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor # 引入更強的模型

# 讀取訓練資料
# 注意：這裡改成你後來給我的正確檔名
df = pd.read_csv('train_car.csv') 

# 查看資料狀況
print(df.shape)
df.head()

# 2.資料預處理

In [ ]:
# --- 修改重點：資料清理與填補，而不是直接刪除 ---

# 1. 處理 'horsepower' 欄位中的異常值 '?'
# 將 '?' 替換為 NaN，然後轉換為數值型態
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')

# 2. 填補缺失值
# 使用 '中位數' 填補 horsepower 的缺失值，避免極端值影響
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# 3. 處理類別特徵 'origin'
# 使用 One-Hot Encoding 將 origin (1, 2, 3) 轉換為多個欄位
df = pd.get_dummies(df, columns=['origin'], prefix='origin')

# 檢查清理後的資料
print("資料預處理完成，目前缺失值數量：")
print(df.isnull().sum().sum())
df.head()

# 3.特徵工程

In [ ]:
# --- 修改重點：選用所有有用的特徵 ---

# 定義要用來預測的特徵欄位
# 我們加入了 origin 轉換後的欄位，並保留了其他數值特徵
# 'name' 是文字名稱，對預測幫助較小且難處理，先排除
# 'mpg' 是答案，當然要去掉
# 'id' 只是編號，去掉

features = ['cylinders', 'displacement', 'horsepower', 'weight', 
            'acceleration', 'model_year', 'origin_1', 'origin_2', 'origin_3']

target = 'mpg'

X = df[features]
y = df[target]

print(f"使用的特徵: {features}")

# 4. 訓練模型

In [ ]:
# 分割訓練集與驗證集 (80% 訓練, 20% 驗證)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# --- 修改重點：使用 Gradient Boosting Regressor 替代原本的 LinearRegression ---
# 這是一個集成模型，能捕捉非線性關係，準確度通常比線性回歸高很多
model = GradientBoostingRegressor(n_estimators=1000, 
                                  learning_rate=0.05, 
                                  max_depth=5, 
                                  random_state=42)

# 訓練模型
model.fit(X_train, y_train)

# 預測驗證集
y_pred = model.predict(X_val)

# 計算 RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f'Validation RMSE: {rmse}')
# 目標是低於 2.69 (Medium Baseline)，這個模型通常可以跑到 2.3~2.5 左右

模型已訓練完成！
訓練誤差 (Train RMSE): 3.4119 MPG
測試誤差 (Test RMSE):  3.8504 MPG

--- 模型學到的關係 ---
特徵 'weight' 的權重: -0.0067
特徵 'acceleration' 的權重: 0.0325
特徵 'model_year' 的權重: 0.7999
特徵 'cylinders' 的權重: 0.3731
特徵 'displacement' 的權重: -0.0031
特徵 'horsepower' 的權重: -0.0095


# 5.輸出提交檔案

In [ ]:
# 讀取測試檔案
df_test = pd.read_csv("test_car.csv")

# --- 測試資料也必須做一樣的預處理 (重要！) ---
# 1. 處理 horsepower
df_test['horsepower'] = pd.to_numeric(df_test['horsepower'], errors='coerce')
df_test['horsepower'] = df_test['horsepower'].fillna(df['horsepower'].median()) # 注意：用訓練集的中位數來填補

# 2. 處理 origin (One-Hot)
df_test = pd.get_dummies(df_test, columns=['origin'], prefix='origin')

# 3. 確保測試集的欄位跟訓練集完全一樣 (避免因為測試集剛好缺某個產地而報錯)
for col in features:
    if col not in df_test.columns:
        df_test[col] = 0

# 產生預測
test_pred = model.predict(df_test[features])

# 建立提交檔案
submission = pd.DataFrame({
    'id': df_test['id'],
    'mpg': test_pred
})

# 存檔
submission.to_csv('submission.csv', index=False)
print("submission.csv 已生成，快去上傳！")
submission.head()

提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：__________ 學號：__________

第一部分：準確度分數 (Accuracy Scores) (1分)  
我的準確度分數：__________  

第二部分：我的實驗記錄 (My Experiment Log) (3分)  
請記錄你做了哪些嘗試來提升分數，至少記錄兩次不同的嘗試。  
【實驗 1】  
    我做的修改：__________________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：__________________________________________________________________  
    該次實驗分數： ____________  
【實驗 2】  
    我做的修改：__________________________________________________________________________________  
    結果與觀察 (分數變化、心得等)：__________________________________________________________________  
    該次實驗分數： ____________  

第三部分：總結與心得 (Conclusion & Reflection) (2分)  
請撰寫一段約 50-100 字的心得總結。內容需包含：  
(1) 你認為本次實驗中，提升準確率最有效的修改是什麼。  
(2) 這次不斷嘗試與修正的過程，帶給你最大的學習與啟發。  
內容：______________________________________________    